# **Importing necessary Libraries**

---



---



In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# **Importing MNIST/Fashion_MNIST dataset**

---



---



In [4]:
(X_train,Y_train),(X_test,Y_test)=tf.keras.datasets.mnist.load_data()
(fX_train,fY_train),(fX_test,fY_test)=tf.keras.datasets.fashion_mnist.load_data()

# **Normalizing Pixel Values between 0-1**

---



---



In [5]:
X_train=np.array(X_train)/255
X_test=np.array(X_test)/255
fX_train=np.array(fX_train)/255
fX_test=np.array(fX_test)/255

# **Defining FFN model Using Subclassing method**

---



---



In [8]:
class FFN(tf.keras.models.Model):
  def __init__(self):
    super(FFN,self).__init__()
    self.flatten=tf.keras.layers.Flatten()
    self.dense=tf.keras.layers.Dense(784,activation='relu')
    self.dense2=tf.keras.layers.Dense(512,activation='relu')
    self.dense3=tf.keras.layers.Dense(256,activation='relu')
    self.dense4=tf.keras.layers.Dense(10,activation='softmax')
  def call(self,x):
    x=self.flatten(x)
    x=self.dense(x)
    x=self.dense2(x)
    x=self.dense3(x)
    Output=self.dense4(x)
    return Output

# **Initializing and compiling model**

---



---



In [9]:
classes=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]
FMNISTmodel=FFN()
MNISTmodel=FFN()
FMNISTmodel.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
MNISTmodel.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# **Training of model**

---



---



In [12]:
print("\nTraining model on MNIST:\n")
MNISTmodel.fit(X_train,Y_train,epochs=10)
print("\nTraining model on Fasion_MNIST:\n")
FMNISTmodel.fit(fX_train,fY_train,epochs=10)


Training model on MNIST:

Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0228 - accuracy: 0.9937
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0213 - accuracy: 0.9945
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0181 - accuracy: 0.9958
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0168 - accuracy: 0.9956
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0158 - accuracy: 0.9962
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0183 - accuracy: 0.9953
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0149 - accuracy: 0.9963
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0148 - accuracy: 0.9966
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0139 - accuracy: 0.9963
Epoch 10/10
1875/1875 [==============================] 

# **Evaluating Accuracy on test dataset**

---



---



In [14]:
print("\n Evaluating 'MNIST test' Accuray and loss\n")
MNISTmodel.evaluate(X_test,Y_test)
print("\n Evaluating 'Fashion_MNIST test' Accuray and loss\n")
FMNISTmodel.evaluate(fX_test,fY_test)


 Evaluating 'MNIST test' Accuray and loss

313/313 [==============================] - 1s 2ms/step - loss: 0.1476 - accuracy: 0.9825

 Evaluating 'Fashion_MNIST test' Accuray and loss

313/313 [==============================] - 1s 2ms/step - loss: 0.3372 - accuracy: 0.8893


[0.3371577262878418, 0.8892999887466431]